In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from collections import Counter

In [23]:
df_energy = pd.read_excel(os.getcwd() + "/Datasets/Ontario/SSC2020_hourly_demand.xlsx")

In [4]:
df_energy

,Date,Hour,Total Energy Use from Electricity (MW),Year,Month
0,2003-01-01,1,14745,2003,1
1,2003-01-01,2,14280,2003,1
2,2003-01-01,3,13821,2003,1
3,2003-01-01,4,13239,2003,1
4,2003-01-01,5,13236,2003,1
...,...,...,...,...,...
122731,2016-12-31,20,16260,2016,12
122732,2016-12-31,21,15658,2016,12
122733,2016-12-31,22,15195,2016,12
122734,2016-12-31,23,14758,2016,12


In [16]:
energy = df_energy["Total Energy Use from Electricity (MW)"]
energy = list(energy)

In [5]:
df_weather = pd.read_excel(os.getcwd() + "/Datasets/Ontario/ssc2020_hourly_weather.xlsx")

In [6]:
df_weather

,time,precipitation,temperature,irradiance_surface,irradiance_toa,snowfall,snow_depth,cloud_cover,air_density
0,2003-01-01 00:00:00,0.0100,-1.720,0.0000,0.0000,0.0011,17.4309,0.3196,1.2612
1,2003-01-01 01:00:00,0.0022,-2.045,0.0000,0.0000,0.0010,17.4307,0.3167,1.2644
2,2003-01-01 02:00:00,0.0014,-2.402,0.0000,0.0000,0.0009,17.4304,0.2958,1.2669
3,2003-01-01 03:00:00,0.0013,-2.732,0.0000,0.0000,0.0008,17.4302,0.3745,1.2695
4,2003-01-01 04:00:00,0.0011,-3.083,0.0000,0.0000,0.0009,17.4300,0.6073,1.2724
...,...,...,...,...,...,...,...,...,...
122731,2016-12-31 19:00:00,0.1642,0.551,65.7947,394.8765,0.0845,16.9569,0.9761,1.2408
122732,2016-12-31 20:00:00,0.1857,0.455,29.8108,238.0150,0.0863,16.9597,0.9729,1.2407
122733,2016-12-31 21:00:00,0.2322,0.180,4.8503,52.9368,0.1055,16.9680,0.9628,1.2417
122734,2016-12-31 22:00:00,0.2462,-0.047,0.0047,0.0378,0.1173,16.9865,0.9353,1.2430


In [19]:
df_weather["Total Energy Use from Electricity (MW)"] = energy

In [20]:
df_weather

,time,precipitation,temperature,irradiance_surface,irradiance_toa,snowfall,snow_depth,cloud_cover,air_density,Total Energy Use from Electricity (MW)
0,2003-01-01 00:00:00,0.0100,-1.720,0.0000,0.0000,0.0011,17.4309,0.3196,1.2612,14745
1,2003-01-01 01:00:00,0.0022,-2.045,0.0000,0.0000,0.0010,17.4307,0.3167,1.2644,14280
2,2003-01-01 02:00:00,0.0014,-2.402,0.0000,0.0000,0.0009,17.4304,0.2958,1.2669,13821
3,2003-01-01 03:00:00,0.0013,-2.732,0.0000,0.0000,0.0008,17.4302,0.3745,1.2695,13239
4,2003-01-01 04:00:00,0.0011,-3.083,0.0000,0.0000,0.0009,17.4300,0.6073,1.2724,13236
...,...,...,...,...,...,...,...,...,...,...
122731,2016-12-31 19:00:00,0.1642,0.551,65.7947,394.8765,0.0845,16.9569,0.9761,1.2408,16260
122732,2016-12-31 20:00:00,0.1857,0.455,29.8108,238.0150,0.0863,16.9597,0.9729,1.2407,15658
122733,2016-12-31 21:00:00,0.2322,0.180,4.8503,52.9368,0.1055,16.9680,0.9628,1.2417,15195
122734,2016-12-31 22:00:00,0.2462,-0.047,0.0047,0.0378,0.1173,16.9865,0.9353,1.2430,14758


In [21]:
# pretty good - no null values!!!
df_weather.isnull().sum()

time                                      0
precipitation                             0
temperature                               0
irradiance_surface                        0
irradiance_toa                            0
snowfall                                  0
snow_depth                                0
cloud_cover                               0
air_density                               0
Total Energy Use from Electricity (MW)    0
dtype: int64

In [22]:
df_weather.to_csv('Hourly_Electricity_Ontario.csv', index=False)